In [4]:
import requests
import requests_cache
import time
from bs4 import BeautifulSoup as bs
import html5lib
import re
import sqlite3
from datetime import datetime


def download(url, num_retries=5):
    requests_cache.install_cache()
    print('Downloading:', url)
    page = None
    try:
        response = requests.get(url)
        # print(response.status_code)
        page = response.text
        if response.status_code >= 400:
            print('Download error:', response.text)
            time.sleep(30)
            print('Tentando novamento: ' + str(num_retries))
            if num_retries and 500 <= response.status_code < 600:
                return download(url, num_retries - 1)
    except requests.exceptions.RequestException as e:
        print('Download error:', e)
    return page


def busca_dados(url):
    html = download(url)
    soup = bs(html, 'html5lib')
    cidade = soup.find('h1', id='momento-localidade').get_text()
    temperatura = soup.find('p', id='momento-temperatura').get_text()
    sensacao = soup.find('li', id='momento-sensacao').get_text()
    umidade = soup.find('li', id='momento-humidade').get_text()
    pressao = soup.find('li', id='momento-pressao').get_text()
    vento = soup.find('a', id='momento-vento').get_text().strip("\n")
    vento = re.sub(r'[^\dkm/h]','',vento)
    atualizado = soup.find('p', id='momento-atualizacao').get_text()
    atualizado = re.search(r'.\d\d:\d\d$',atualizado).group(0)
    dados = [(cidade, temperatura, sensacao, umidade, pressao, vento, atualizado)]
    return dados

def exibe():
    url = 'https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi'
    res = busca_dados(url)
    for i in res:
        texto = """
        Cidade:      {0}
        Temperatura: {1}
        Sensação:    {2}
        Umidade:     {3}
        Pressão:     {4}
        Vento:       {5}
        Atualizado:  {6}
        """.format(res[0], res[1], res[2], res[3], res[4], res[5], res[6])
    print(texto)

def main():
    exibe()

if __name__=='__main__':
    main()

Downloading: https://www.climatempo.com.br/previsao-do-tempo/cidade/264/teresina-pi


IndexError: list index out of range